In [14]:
import torch
from tqdm.notebook import tqdm

In [15]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, filename):
        self.data = []
        with open(filename, 'r') as f:
            for line in f:
                numbers = [float(num) for num in line.strip().split()]
                self.data.append((numbers[:4], numbers[4:]))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        inputs, targets = self.data[idx]
        return torch.tensor(inputs), torch.tensor(targets)

In [16]:
filename = "data/sph_100_10_20.txt"
full_dataset = CustomDataset(filename)

train_size = int(0.8 * len(full_dataset))
rest_size = len(full_dataset) - train_size
val_size = rest_size // 2
test_size = rest_size // 2
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size, test_size])

In [17]:
from torch.utils.data import DataLoader

BATCH_SIZE = 32
train_shuffle = True
val_shuffle = False

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=train_shuffle)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=val_shuffle)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [18]:
for inputs, targets in train_dataloader:
    assert inputs.shape[1] == 4 and targets.shape[1] == 2

In [19]:
import torch.nn as nn
import torch.nn.init as init

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(4, 128),
            nn.Tanh(),
            nn.Linear(128, 128),
            nn.Tanh(),
            nn.Linear(128, 2)
        )
        self.__init_weights()

    def __init_weights(self):
        # Initialize linear layers using Kaiming (He) uniform initialization
        for m in self.layers.children():
            if isinstance(m, nn.Linear):
                init.kaiming_uniform_(m.weight, mode='fan_in', nonlinearity='tanh')
                if m.bias is not None:
                    init.zeros_(m.bias)

    def forward(self, x):
        return self.layers(x)

In [20]:
LR = 5e-5
NUM_EPOCHS = 50000
MOMENTUM = 0.9
WEIGHT_DECAY = 0.999

In [21]:
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

model = MLP()
loss_fn = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=WEIGHT_DECAY)
scheduler = lr_scheduler.LinearLR(
    optimizer,
    start_factor=1.0,
    end_factor=0.3,
    total_iters=NUM_EPOCHS / 0.20)

In [22]:
from torch.utils.tensorboard import SummaryWriter

def train(model, dataloader, optimizer, scheduler, loss_fn, epoch, writer, log_perc = 0.1):
    model.train()
    total_loss = 0
    total_diff = 0

    logs_steps = int(log_perc * len(dataloader))
    start_step = epoch * len(dataloader)

    before_lr = optimizer.param_groups[0]["lr"]
    writer.add_scalar('Lr/Train', before_lr, epoch)
    for idx, (inputs, targets) in enumerate(dataloader):
        optimizer.zero_grad()

        outputs = model(inputs)

        loss = loss_fn(outputs, targets)
        diff = torch.abs(outputs - targets).mean()

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_diff += diff.item()
        
        if idx % logs_steps == 0:
            writer.add_scalar('Loss/Train', loss.item(), start_step + idx)
            writer.add_scalar('Absolute Difference/Train', diff.item(), start_step + idx)

    scheduler.step()
    after_lr = optimizer.param_groups[0]["lr"]
    average_loss = total_loss / len(dataloader)
    average_diff = total_diff / len(dataloader)

    writer.add_scalar('Avg Loss/Train', average_loss, epoch)
    writer.add_scalar('Avg Absolute Difference/Train', average_diff, epoch)
    writer.add_scalar('Lr/Train', after_lr, epoch)

    # print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] Train Loss: {average_loss:.4f}, Train Diff: {average_diff:.15f}")

In [23]:
def validate(model, dataloader, loss_fn, epoch, writer):
    model.eval()
    total_loss = 0
    total_diff = 0

    with torch.no_grad():
        for inputs, targets in dataloader:
            outputs = model(inputs)

            loss = loss_fn(outputs, targets)
            diff = torch.abs(outputs - targets).mean()
            
            total_loss += loss.item()
            total_diff += diff.item()

    average_loss = total_loss / len(dataloader)
    average_diff = total_diff / len(dataloader)

    if writer is not None:
        writer.add_scalar('Avg Loss/Val', average_loss, epoch)
        writer.add_scalar('Avg Absolute Difference/Val', average_diff, epoch)

    if epoch is not None:
        print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] Val Loss: {average_loss:.4f}, Val Diff: {average_diff:.15f}")
    else:
        print(f"Test Loss: {average_loss:.4f}, Test Diff: {average_diff:.15f}")

In [24]:
import datetime

now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
writer = SummaryWriter(f"tb_logs/{now}")

VALIDATION_STEPS = NUM_EPOCHS / 0.01

for idx, epoch in enumerate(tqdm(range(NUM_EPOCHS))):
    train(model, train_dataloader, optimizer, scheduler, loss_fn, epoch, writer)

    if idx % VALIDATION_STEPS == 0:
        validate(model, val_dataloader, loss_fn, epoch, writer)

# Launch TensorBoard: `tensorboard --logdir=tb_logs`

  0%|          | 0/50000 [00:00<?, ?it/s]

Epoch [1/50000] Val Loss: 0.0466, Val Diff: 0.153015180623957


KeyboardInterrupt: 

In [ ]:
validate(model, test_dataloader, loss_fn, epoch, writer)

  0%|          | 0/84 [00:00<?, ?it/s]

Epoch [10/10] Val Loss: 0.0405, Val Diff: 0.121164027335388
